[DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

#Respiratory Disease Tracking Dashboard

This dashboard contains two graphs:
- Influenza testing positivity weekly
- RSV admission rates by ages
Use the dropdowns and slider to explore different years and epiweeks.  


This is a template for your DIY Disease Tracking Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [3]:
#Creating a dictionary and loading the file 
jsondata = {}

with open("influenza_positivity.json", "rt") as f:
    jsondata["influenza_positivity"] = json.load(f)



## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [4]:


def wrangle_data(rawdata):
    """Takes raw influenza positivity data from JSON or API and 
    returns a tidy DataFrame for plotting."""
    
    df = pd.DataFrame(rawdata)

    # keep only columns we need
    df = df[["year", "epiweek", "age", "metric_value"]]

    # correct data types
    df["year"] = df["year"].astype(int)
    df["epiweek"] = df["epiweek"].astype(int)

    # sort for safety
    df = df.sort_values(["year", "epiweek"])

    return df



# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df = wrangle_data(jsondata["influenza_positivity"]) # df is the dataframe for plotting
    




In [5]:
#API Wrapper Class 
class APIwrapper:
    # class variables shared among all instances
    _access_point="https://api.ukhsa-dashboard.data.gov.uk"
    _last_access=0.0 # time of last api access
    
    def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
        """ Init the APIwrapper object, constructing the endpoint from the structure
        parameters """
        # build the path with all the required structure parameters. You do not need to edit this line,
        # parameters will be replaced by the actual values when you instantiate an object of the class!
        url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                  f"{geography_type}/geographies/{geography}/metrics/{metric}")
        # our starting API endpoint
        self._start_url=APIwrapper._access_point+url_path
        self._filters=None
        self._page_size=-1
        # will contain the number of items
        self.count=None

    def get_page(self, filters={}, page_size=5):
        """ Access the API and download the next page of data. Sets the count
        attribute to the total number of items available for this query. Changing
        filters or page_size will cause get_page to restart from page 1. Rate
        limited to three request per second. The page_size parameter sets the number
        of data points in one response page (maximum 365); use the default value 
        for debugging your structure and filters. """
        # Check page size is within range
        if page_size>365:
            raise ValueError("Max supported page size is 365")
        # restart from first page if page or filters have changed
        if filters!=self._filters or page_size!=self._page_size:
            self._filters=filters
            self._page_size=page_size
            self._next_url=self._start_url
        # signal the end of data condition
        if self._next_url==None: 
            return [] # we already fetched the last page
        # simple rate limiting to avoid bans
        curr_time=time.time() # Unix time: number of seconds since the Epoch
        deltat=curr_time-APIwrapper._last_access
        if deltat<0.33: # max 3 requests/second
            time.sleep(0.33-deltat)
        APIwrapper._last_access=curr_time
        # build parameter dictionary by removing all the None
        # values from filters and adding page_size
        parameters={x: y for x, y in filters.items() if y!=None}
        parameters['page_size']=page_size
        # the page parameter is already included in _next_url.
        # This is the API access. Response is a dictionary with various keys.
        # the .json() method decodes the response into Python object (dictionaries,
        # lists; 'null' values are translated as None).
        response = requests.get(self._next_url, params=parameters).json()
        # update url so we'll fetch the next page
        self._next_url=response['next']
        self.count=response['count']
        # data are in the nested 'results' list
        return response['results'] 

    def get_all_pages(self, filters={}, page_size=365):
        """ Access the API and download all available data pages of data. Sets the count
        attribute to the total number of items available for this query. API access rate
        limited to three request per second. The page_size parameter sets the number
        of data points in one response page (maximum 365), and controls the trade-off
        between time to load a page and number of pages; the default should work well 
        in most cases. The number of items returned should in any case be equal to 
        the count attribute. """
        data=[] # build up all data here
        while True:
            # use get_page to do the job, including the pacing
            next_page=self.get_page(filters, page_size)
            if next_page==[]:
                break # we are done
            data.extend(next_page)
        return data

In [6]:
# Accessing data : forming a structure dictionary
structure = {
    "theme": "infectious_disease",
    "sub_theme": "respiratory",
    "topic": "Influenza",
    "geography_type": "Nation",
    "geography": "England",
    "metric": "influenza_testing_positivityByWeek"
}

# Creating an APIwrapper object using the above structure
api = APIwrapper(**structure)

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [7]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """Accesses the UKHSA API. Returns raw data in the same struc
    ture as the JSON file."""

    # This retrieves all influenza positivity metric values from the UKHSA API
    apidata = api.get_all_pages(page_size=365)

    return apidata  # return data read from the API


In [8]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def refresh_all_callback(button):

    """ Button callback.
    Accesses API for both metrics, wrangles data, updates global variables df (Metric 1)
    and rsv_df_small (Metric 2), then refreshes both graphs.
    Including simple error handling so the dashboard does not crash if the API fails.
    """
    # Give the user visual feedback while fetching and adding some error handling 
    button.icon = "spinner"
    button.button_style = "warning"
    button.description = "Fetching..."
    
    try:
        # ---------- Metric 1: Influenza (existing template logic) ----------
        apidata = access_api()          # uses your existing access_api()
        
        # wrangle the data and overwrite the dataframe for plotting (Metric 1)
        global df
        df = wrangle_data(apidata)      # your existing wrangle_data(apidata)
        
        # ---------- Metric 2: RSV admissions ----------
        rsvdata = access_rsv_api()      # new function for RSV
        
        # wrangle RSV data and overwrite rsv_df and rsv_df_small used for plotting
        global rsv_df, rsv_df_small
        rsv_df = pd.DataFrame(rsvdata)
        rsv_df_small = rsv_df[["year", "epiweek", "age", "metric_value"]].copy()
        rsv_df_small["year"] = rsv_df_small["year"].astype(int)
        rsv_df_small["epiweek"] = rsv_df_small["epiweek"].astype(int)
        
    
        
        # Force both graphs to redraw
        refresh_graph()       # existing function for Metric 1
        refresh_rsv_graph()   # new helper for Metric 2
        
        # ---------- Success feedback ----------
        button.icon = "check"
        button.button_style = "success"
        button.description = "Updated"

        # button.disabled = True   # optional: disable after one successful update
        
    except Exception as e:
        # ---------- Error handling ----------
        # If anything goes wrong (API down, network error, etc), we:
        # - do NOT overwrite the existing (canned) df / rsv_df_small
        # - inform the user via button appearance
        button.icon = "times"
        button.button_style = "danger"
        button.description = "API error"
        # You can optionally print the error for debugging in Jupyter (won't show in Voila)
        print("Error while fetching data from API:", e)

    
apibutton=wdg.Button(
    description='Refresh All Data', # you may want to change this...
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Fetch Latest Data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(refresh_all_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='warning', description='Refresh All Data', icon='download', style=ButtonStyle(), tooltip='…

## Graphs and Analysis

## Metric 1: Influenza Testing Positivity

This graph shows the weekly influenza testing positivity (%) in England for the 05–14 age group.

### How to use:
- Select a **year** from the dropdown menu.
- The line graph updates automatically.
- The x-axis shows epiweeks . 
- The y-axis shows influenza positivity (%).

In [9]:
import ipywidgets as wdg
import matplotlib.pyplot as plt

# We fix one age group, to avoid mixing different ages on the same line
AGE_TO_USE = "05-14"   # you can change this later if needed

def plot_positivity(graphyear):
    """Graph plotting function for influenza positivity"""
    
    # Filter to the chosen age group AND year
    df_year_age = df[(df["age"] == AGE_TO_USE) & (df["year"] == graphyear)]
    
    # Sort by epiweek to make the line smooth
    df_year_age = df_year_age.sort_values("epiweek")
    
    # Plot epiweek on the x-axis and positivity on the y-axis
    plt.figure()
    plt.plot(df_year_age["epiweek"], df_year_age["metric_value"])
    plt.xlabel("Epiweek")
    plt.ylabel("Influenza testing positivity (%)")
    plt.title(f"Influenza testing positivity in England (age {AGE_TO_USE}), {graphyear}")
    plt.show()  # important for interactive updates


# Widget: dropdown to select the year
available_years = sorted(df["year"].unique())

year_selector = wdg.Dropdown(
    options=available_years,
    value=available_years[0],   # or max(available_years) if you prefer the latest year
    description='Year:',
    disabled=False,
)

def refresh_graph():
    """Force a redraw of the graph by simulating a change in the year dropdown."""
    current = year_selector.value
    years = list(year_selector.options)
    if len(years) > 1:
        # pick a different year temporarily
        other = years[0] if current != years[0] else years[1]
        year_selector.value = other   # forces redraw via interactive_output
        year_selector.value = current # change it back


# Connect the plotting function and the widget 
graph = wdg.interactive_output(plot_positivity, {'graphyear': year_selector})

# Display the widget and the graph together
display(year_selector, graph)




Dropdown(description='Year:', options=(np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int6…

Output()

In [10]:
#Metric 2 - RSV Weekly Healthcare Admission Rate by age group 


In [11]:
#HEADING : LOADING INITIAL DATA FROM DISK FOR METRIC 2 

In [12]:
# Essential IMPORTS  for METRIC 2 that are needed at the beginning 
import matplotlib.pyplot as plt
import ipywidgets as wdg
from IPython.display import display
import json
import pandas as pd


In [13]:
#Loading JSON data and creating dataframes for METRIC 2 : RSV Weekly admission rate by Age Group 
import json
import pandas as pd

# Loading Metric 2 raw JSON data that I saved in my other notebook called RSV admissions by age file 
with open("rsv_admissions_by_age.json", "rt") as INF:
    rsv_raw = json.load(INF)

# Converting to DataFrame
rsv_df = pd.DataFrame(rsv_raw)

# Quick check to ensure that data is present 
rsv_df.head()


,theme,sub_theme,topic,geography_type,geography,geography_code,metric,metric_group,stratum,sex,age,year,month,epiweek,date,metric_value,in_reporting_delay_period
0,infectious_disease,respiratory,RSV,Nation,England,E92000001,RSV_healthcare_admissionRateByWeek,healthcare,default,all,65-74,2020,9,40,2020-09-28,0.0,False
1,infectious_disease,respiratory,RSV,Nation,England,E92000001,RSV_healthcare_admissionRateByWeek,healthcare,default,all,05-14,2020,9,40,2020-09-28,0.0,False
2,infectious_disease,respiratory,RSV,Nation,England,E92000001,RSV_healthcare_admissionRateByWeek,healthcare,default,all,75+,2020,9,40,2020-09-28,0.0,False
3,infectious_disease,respiratory,RSV,Nation,England,E92000001,RSV_healthcare_admissionRateByWeek,healthcare,default,all,00-04,2020,9,40,2020-09-28,0.0,False
4,infectious_disease,respiratory,RSV,Nation,England,E92000001,RSV_healthcare_admissionRateByWeek,healthcare,default,all,85+,2020,9,40,2020-09-28,0.0,False


In [14]:
#HEADING : WRANGLE DATA FOR METRIC 2 

In [15]:
#Importing widgets ad cleaning my DF 
import ipywidgets as wdg
from IPython.display import display

# Keep only the columns we need for plotting Metric 2
rsv_df_small = rsv_df[["year", "epiweek", "age", "metric_value"]].copy()

# Make sure year and epiweek are numeric.
rsv_df_small["year"] = rsv_df_small["year"].astype(int)
rsv_df_small["epiweek"] = rsv_df_small["epiweek"].astype(int)

#  Widgets for Metric 2 (similar style to Metric 1) 

# Year dropdown (like Metric 1)
rsv_years = sorted(rsv_df_small["year"].unique())
year_selector_rsv = wdg.Dropdown(
    options=rsv_years,
    value=2021,      # choosing 2021 year by default 
    description="Year:",
    disabled=False,
)

# Epiweek slider - using a different widget to meet the instructions given by prof 
min_week = int(rsv_df_small["epiweek"].min())
max_week = int(rsv_df_small["epiweek"].max())
week_selector_rsv = wdg.IntSlider(
    value=13,        
    min=min_week,
    max=max_week,
    step=1,
    description="Epiweek:",
    continuous_update=False,
)

# displaying the widgets to check they appear
display(year_selector_rsv, week_selector_rsv)


Dropdown(description='Year:', index=1, options=(np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)…

IntSlider(value=13, continuous_update=False, description='Epiweek:', max=52, min=1)

In [16]:
#HEADING : GRAPHS AND ANALYSIS FOR METRIC 2 

## Metric 2 : RSV Hospital Admission Rate by Age Group 
The graph displays weekly RSV admissions in England broken by age groups . 
The graph shows that each bar represents an age group and the height shows the admission rate for the week . 
To use this graph, use the year drop down to select a year. Choose the slider to choose a specific week. The bar chart updates automatically when you change either widget. 

In [17]:
import matplotlib.pyplot as plt

def plot_rsv_admissions(graphyear, graphepiweek):
    """Graph plotting function for RSV admission rate by age group"""
    
    # Filter for selected year and epiweek
    df_sel = rsv_df_small[
        (rsv_df_small["year"] == graphyear) &
        (rsv_df_small["epiweek"] == graphepiweek)
    ]
    
    if df_sel.empty:
        print(f"No data available for year {graphyear}, epiweek {graphepiweek}.")
        return
    
    df_sel = df_sel.sort_values("age")
    
    df_sel.plot(
        kind="bar",
        x="age",
        y="metric_value",
        legend=False
    )
    plt.xlabel("Age group")
    plt.ylabel("RSV healthcare admission rate")
    plt.title(f"RSV admission rate by age group\nEngland, {graphyear}, epiweek {graphepiweek}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [18]:
#HEADING : DISPLAY GRAPH FOR METRIC 2 

In [19]:
#Connecting Metric 2 plotting function and the widgets

rsv_graph = wdg.interactive_output(
    plot_rsv_admissions,
    {
        'graphyear': year_selector_rsv,
        'graphepiweek': week_selector_rsv
    }
)

# Displaying the Metric 2 widgets and graph together
display(year_selector_rsv, week_selector_rsv, rsv_graph)


Dropdown(description='Year:', index=1, options=(np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)…

IntSlider(value=13, continuous_update=False, description='Epiweek:', max=52, min=1)

Output()

## REFRESH DATA AND GRAPHS 
This refresh button button refreshes both graphs and lets you know if the UKHSA API is unavailable, but will continue working . 

In [20]:
def refresh_rsv_graph():
    """
    Force a redraw of the RSV graph by simulating a change in the RSV year selector.
    This follows the same pattern as refresh_graph() in Metric 1.
    """
    current = year_selector_rsv.value
    years = list(year_selector_rsv.options)

    if len(years) > 1:
        # Temporarily switch to a different year to trigger the interactive_output update
        other = years[0] if current != years[0] else years[1]
        year_selector_rsv.value = other    # forces redraw
        year_selector_rsv.value = current  # change it back


## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."